In [1]:
import sys
import os
import time

sys.path.insert(0, '..')
sys.path.insert(0, '../../futile/python')

package = 'RemoteRunner'

import BigDFT
BigDFT.Logger.path = os.path.join(os.getcwd(), f'BigDFT_test{package}.yaml')
BigDFT.Logger.level = 'debug'
BigDFT.Logger.overwrite = True
BigDFT.Logger.info(f'{package} unit testing begins')

import BigDFT.RemoteRunners as r

from BigDFT.URL import URL

remote_dir = os.path.join(os.getcwd(), f'fake_remote_{package}')
local_dir = os.path.join(os.getcwd(), f'fake_local_{package}')

print(f'running with:\nremote dir: {remote_dir}\nlocal dir: {local_dir}')

In [2]:
maxtime = 20
def remote_func(arg):
    return arg

In [3]:
BigDFT.Logger.info('checking a basic run')
url = URL(verbose=True)
url.cmd(f'rm -r {local_dir}/*', suppress_warn=True)
url.cmd(f'rm -r {remote_dir}/*', suppress_warn=True)
url.test_connection(verbose=False)

remote_run = r.RemoteRunner(function=remote_func,
                            url=url,                              
                            remote_dir=remote_dir,
                            local_dir=local_dir,
                            script='#!/bin/bash',
                            arguments=dict(arg='Hello world'),
                            verbose=True,
                            skip=False)

remote_run.run()

while not remote_run.is_finished(anyfile=False, timeout=maxtime):
    time.sleep(1)
result = remote_run.fetch_result()

cmd returned: 
cmd returned: 
cmd returned: 
cmd returned: 
sending ['remote_func-run.sh', 'remote_func-function-files.tar.gz', 'remote_func-function-run.py']
from /home/test/bigdft-suite/PyBigDFT/Tests/fake_local_RemoteRunner to /home/test/bigdft-suite/PyBigDFT/Tests/fake_remote_RemoteRunner
cmd returned: sending incremental file list
remote_func-function-files.tar.gz
remote_func-function-run.py
remote_func-run.sh

sent 2,158 bytes  received 73 bytes  4,462.00 bytes/sec
total size is 1,905  speedup is 0.85

Checking for finished run... locally... cmd returned: remote_func-function-files.tar.gz
remote_func-function-run.py
remote_func-run.sh

No (No result file)
Checking for finished run... locally... cmd returned: remote_func-function-files.tar.gz
remote_func-function-result.json
remote_func-function-run.py
remote_func-run.sh

Yes (And recent run)
Checking for finished run... locally... cmd returned: remote_func-function-files.tar.gz
remote_func-function-result.json
remote_func-functio

In [4]:
# NBVAL_CHECK_OUTPUT
# check a basic run

print(result)

Hello world


In [5]:
BigDFT.Logger.info('testing skip=True')
remote_run = r.RemoteRunner(function=remote_func,
                            url=url,                              
                            remote_dir=remote_dir,
                            local_dir=local_dir,
                            script='#!/bin/bash',
                            arguments=dict(arg="Don't print this"),
                            verbose=True,
                            rsync='auv')
# initial run should print old result
remote_run.run(skip=True, asynchronous=False)
idx = 0
while not remote_run.is_finished(anyfile=False):
    time.sleep(1)
    idx += 1
    if idx >= maxtime:
        raise ValueError('search timed out!')
result1 = remote_run.fetch_result()

BigDFT.Logger.info('testing force=True')
# followup run should force new result through
# delay is required or the "results stealing" bug will rear its ugly head
time.sleep(5)
remote_run.run(force=True, asynchronous=False)
while not remote_run.is_finished(anyfile=False, timeout=maxtime):
    time.sleep(1)
result2 = remote_run.fetch_result()

cmd returned: 
cmd returned: 
cmd returned: remote_func-function-files.tar.gz
remote_func-function-result.json
remote_func-function-run.py
remote_func-run.sh

cmd returned: 
Checking for finished run... locally... cmd returned: remote_func-function-files.tar.gz
remote_func-function-result.json
remote_func-function-run.py
remote_func-run.sh

Yes (Found a results file)
sending ['remote_func-function-result.json']
from /home/test/bigdft-suite/PyBigDFT/Tests/fake_remote_RemoteRunner to /home/test/bigdft-suite/PyBigDFT/Tests/fake_local_RemoteRunner
cmd returned: sending incremental file list

sent 83 bytes  received 12 bytes  190.00 bytes/sec
total size is 13  speedup is 0.14

Checking for finished run... locally... cmd returned: remote_func-function-files.tar.gz
remote_func-function-result.json
remote_func-function-run.py
remote_func-run.sh

Yes (And recent run)
Checking for finished run... locally... cmd returned: remote_func-function-files.tar.gz
remote_func-function-result.json
remote_f

In [6]:
# NBVAL_CHECK_OUTPUT
# check that skip works
print(result1)
print(result2)

Hello world
Don't print this


is_finished tests to follow

In [7]:
def func_with_delay(t):
    import time
    time.sleep(t)
    return t

internal_delay = 3  # delay for function
external_delay = 6  # delay to wait for result

url = URL(verbose=False)
url.test_connection()
BigDFT.Logger.info(f'testing is_finished')
url.cmd(f'rm -r {local_dir}/*', suppress_warn=True)
url.cmd(f'rm -r {remote_dir}/*', suppress_warn=True)
delayrunner = r.RemoteRunner(function=func_with_delay,
                             url=url,                              
                             remote_dir=remote_dir,
                             local_dir=local_dir,
                             script='#!/bin/bash',
                             arguments=dict(t=internal_delay),
                             verbose=True,
                             skip=False)


######## local permissions ########
creating test folder /URL_connection_test
	permission denied, trying again in /home/test
creating test folder /home/test/URL_connection_test
	success, cleaning up

######## local permissions ########
creating test folder /URL_connection_test
	permission denied, trying again in /home/test
creating test folder /home/test/URL_connection_test
	success, cleaning up

######## file push test ########
test push testfile_push from /home/test/rsync_test_local to /home/test/rsync_test_local
	success, cleaning up

######## file pull test ########
test pull testfile_pull from /home/test/rsync_test_remote to /home/test/rsync_test_local
	success, cleaning up
used 26 cmd calls for this test, taking 0.11 seconds (averaging 0.00s per call)


In [8]:
# NBVAL_CHECK_OUTPUT
BigDFT.Logger.info('basic run and check')
delayrunner.run(asynchronous=False)
while not delayrunner.is_finished(timeout=maxtime):
    time.sleep(external_delay)
print(delayrunner.fetch_result())

Checking for finished run... locally... Yes (Found a results file)
Checking for finished run... locally... Yes (Found a results file)
Checking for finished run... locally... Yes (Found a results file)
3


In [9]:
# NBVAL_CHECK_OUTPUT
BigDFT.Logger.info('testing with skip=False (rerun check)')
time.sleep(1)
delayrunner.run(skip=False, asynchronous=False)
while not delayrunner.is_finished(anyfile=False, timeout=maxtime):
    time.sleep(external_delay)
print(delayrunner.fetch_result())

Checking for finished run... locally... Yes (Found a results file)
Checking for finished run... locally... Yes (And recent run)
Checking for finished run... locally... Yes (Found a results file)
3


In [10]:
# NBVAL_CHECK_OUTPUT
BigDFT.Logger.info('checking with skip=True (pickup check)')
time.sleep(1)
delayrunner.run(skip=True, asynchronous=False)
while not delayrunner.is_finished(anyfile=False, timeout=maxtime):
    time.sleep(external_delay)
print(delayrunner.fetch_result())

Checking for finished run... locally... Yes (Found a results file)
Checking for finished run... locally... Yes (And recent run)
Checking for finished run... locally... Yes (Found a results file)
3


In [11]:
# NBVAL_CHECK_OUTPUT
# should time out
BigDFT.Logger.info('checking that a run without results correctly times out')
url.cmd(f'rm -r {remote_dir}/*', suppress_warn=True)
try:
    while not delayrunner.is_finished(anyfile=False, timeout=3):
        time.sleep(1)
except ValueError as E:
    if "timed out" in str(E):
        print('expected timeout error')
    else:
        raise E

Checking for finished run... locally... No (No result file)
Checking for finished run... locally... No (No result file)
Checking for finished run... locally... No (No result file)
expected timeout error


# Check that a runner cannot wait for itself

In [12]:
BigDFT.Logger.info('testing runner.wait')

def simple_func():
    return 'finished!'

url.cmd(f'rm -r {local_dir}/*', suppress_warn=True)
url.cmd(f'rm -r {remote_dir}/*', suppress_warn=True)
delayrunner = r.RemoteRunner(function=func_with_delay,
                             url=url,                              
                             remote_dir=remote_dir,
                             local_dir=local_dir,
                             script='#!/bin/bash',
                             arguments=dict(t=internal_delay),
                             verbose=True,
                             skip=False)

secondaryrunner = r.RemoteRunner(function=simple_func,
                                 url=url,                              
                                 remote_dir=remote_dir,
                                 local_dir=local_dir,
                                 script='#!/bin/bash',
                                 verbose=True,
                                 skip=False)
secondaryrunner.wait(delayrunner)

In [13]:
# NBVAL_CHECK_OUTPUT
delayrunner.run(skip=False, force=True)
secondaryrunner.run()

Checking for finished run... locally... No (No result file)
previous run is not complete! Skipping this run


In [14]:
# NBVAL_CHECK_OUTPUT
time.sleep(internal_delay * 2)
secondaryrunner.run()

Checking for finished run... locally... Yes (Found a results file)


In [15]:
# NBVAL_CHECK_OUTPUT
time.sleep(1)
secondaryrunner.fetch_result()

Checking for finished run... locally... Yes (Found a results file)


'finished!'

In [16]:
BigDFT.Logger.info('testing that a run will not wait for itself')

runner = r.RemoteRunner(function = simple_func)

In [17]:
# NBVAL_CHECK_OUTPUT
try:
    runner.wait(runner, anyfile=False)

    runner.run()
    print('runner has run successfully..?')
except ValueError as E:
    print(E)

awaiting self would render the runner unable to run!


# Test that CallableAttrDict scripts behave

In [18]:
from BigDFT.RemoteRunnerUtils import CallableAttrDict

class local(CallableAttrDict):
    def __init__(self):
        self.submitter = 'bash'  # this should be ignored by the runner
        
    def __str__(self):
        return "#!/bin/bash\n"
    
script = local()

In [19]:
# NBVAL_CHECK_OUTPUT

# inputs should be ignored, and run as expected
runner = r.RemoteRunner(function = simple_func,
                        submitter = 'fail',  # ordinarily this would not work
                        script = script)

runner.run()
time.sleep(1)
runner.fetch_result()

extracting attributes from callable script
Checking for finished run... locally... Yes (Found a results file)


'finished!'

In [20]:
# NBVAL_CHECK_OUTPUT
try:
    runner = r.RemoteRunner(function = simple_func,
                            script = local)
except ValueError as E:
    if "script has not been called" in str(E):
        print('failed as expected')

failed as expected
